In [1]:
import random
import string
import time
import tensorflow as tf
import numpy as np
from scipy import sparse

In [25]:
list_of_strs = [''.join([string.ascii_lowercase[random.randrange(0, 26)] for i in range(random.randrange(3, 5))])
                for j in range(500)]

In [26]:
list_of_chars = set.union(*[set(i) for i in list_of_strs])

In [27]:
char_dict = {value:idx for idx, value in enumerate(sorted(list_of_chars))}

In [28]:
char_dict[' '] = max(char_dict.values())+1

In [29]:
max_length = max([len(i) for i in list_of_strs])

In [30]:
def onehot_string(x, max_length):
    
    if x!=x:
        return None
    
    #Pad to max length
    while len(x)<max_length:
        x+=' '
        
    sparse_matrix = sparse.hstack([sparse.csr_matrix([0 if char_dict[current_char]!=i else 1 for i in range(len(char_dict))]) for current_char in x])
    
    return sparse_matrix

In [31]:
n_tiles = 500 # dense_strs.shape[0]

In [32]:
start_time = time.time()
sparse_strs = sparse.vstack([onehot_string(str_, max_length) for str_ in list_of_strs])
end_time = time.time()

In [33]:
dense_strs = sparse_strs.toarray()

In [34]:
max_length = dense_strs.shape[1]

In [35]:
dense_strs.shape

(500, 108)

In [36]:
vector_data = tf.placeholder(dtype=tf.float32, shape=dense_strs.shape)

In [37]:
tiled_vector_data = tf.reshape(tf.tile(vector_data, multiples=[1, n_tiles]), 
                               shape=[dense_strs.shape[0], n_tiles, dense_strs.shape[1]])

In [38]:
tiled_vector_data

<tf.Tensor 'Reshape_3:0' shape=(500, 500, 108) dtype=float32>

In [17]:
tile_ref_data = tf.reshape(tf.tile(vector_data, multiples=[n_tiles, 1]), 
                           shape=[dense_strs.shape[0], n_tiles, dense_strs.shape[1]])

In [18]:
not_equal = tf.cast(tf.equal(tile_ref_data, tiled_vector_data), dtype=tf.float32)

In [19]:
dist_sum = tf.reduce_sum(not_equal, axis=-1)

In [20]:
nearest_neighbors = tf.nn.top_k(dist_sum, k=2, sorted=True)

In [21]:
_, hamming_top_args = tf.nn.top_k(nearest_neighbors.values[:,1], k=nearest_neighbors.values.shape[0])

In [22]:
tile_ref_data

<tf.Tensor 'Reshape_1:0' shape=(500, 500, 513) dtype=float32>

In [302]:
#nearest_neighbors.values[hamming_top_args]

In [303]:
with tf.Session() as sess:
    t = sess.run([hamming_top_args, nearest_neighbors.indices, nearest_neighbors.values], {vector_data:dense_strs})

In [313]:
def map_nearest_neighbor(string_list, top_args, top_values):
    
    if type(string_list)!=np.array:
        string_list = np.array(string_list)
    
    results = []
    values = []
    for idx, top_arg in enumerate(top_args):
        results.append(string_list[top_arg])
        values.append(top_values[idx])
    return np.array(results), np.array(values)

In [314]:
nn_results = map_nearest_neighbor(list_of_strs, t[1][t[0]], t[2][t[0]])